# Loading the Log Data and Preprocessing

In [ ]:
import re
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')
from textblob import TextBlob
import pickle as p
object_file = p.load(open("results_rev1.pkl",'rb'))

results = []
for item in object_file:
    results.append(item)

In [218]:
data_logs = [' '.join(result) for result in results[-100000:]]

# Looking at the Adj and Verb Tags

In [219]:
sentences = data_logs #tokenize sentences
nouns = [] #empty to array to hold all nouns

for sentence in sentences:
    noun = []
    for word,pos in nltk.pos_tag(nltk.word_tokenize(str(sentence))):
        if (pos == 'VB' or pos == 'VBD' or pos == 'VBG' or pos == 'VBN' or pos == 'VBP' or pos == 'VBZ' or pos == 'JJ'):
             noun.append(word)
    nouns.append(noun)

In [220]:
from collections import defaultdict, Counter
counter = Counter()
my_dict = defaultdict(list) 
for i,(log, noun) in enumerate(zip(data_logs,nouns)):
    for word in noun:
        counter[word]+=1
    key = ' '.join(noun)
    my_dict[key].append(log)

In [210]:
# my_dict_ = defaultdict(list) 
# for i,(log, noun) in enumerate(zip(data_logs,nouns)):
#     word = []
#     for w in noun:
#         if counter[w]>10:
#             word.append(w)
#     key = ' '.join(word)
#     my_dict_[key].append(log)

In [249]:
trivial_act_tokens = ['open',
'run',
'start' ,
'stop',
'cmd',
'ping',
'cancele', 'waite' ,
'sendmail', 'dbus-send' ,
'test' ,
'begin',
'root',
'update',
'get' ,
'trust',
'reload',
'adapt',
'skip',
'execute',
'reload',
'install',
'detect',
'close',
'save',
'determine',
'replace']

## counting each key/category

In [251]:
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
categories_lengths = []
my_dict_trivial = defaultdict(list) 

for key in list(my_dict.keys()):
    if len(my_dict[key])<1000:
        t_key_list = []
        for word in word_tokenize(key):
            if lemmatizer.lemmatize(word) in trivial_act_tokens:    
                t_key_list.append(lemmatizer.lemmatize(word))
        key_=' '.join(t_key_list)
        my_dict_trivial[key_]+=my_dict[key]
    else:
        my_dict_trivial[key]=my_dict[key]

                
                

#    categories_lengths.append((len(my_dict[key]), key))    

[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [255]:
for key_ in my_dict_trivial.keys():
    print(len(my_dict_trivial[key_]))

14604
13
30
14004
2
42809
2
14004
88
2
15
20
14006
41
1
353
4
2


In [247]:
len(my_dict['']), my_dict[''] 

(14029,
 ['dhclient bound renewal seconds',
  'dhclient bound renewal seconds',
  'dhclient bound renewal seconds',
  'dhclient bound renewal seconds',
  'dhclient bound renewal seconds',
  'dhclient bound renewal seconds',
  'dhclient bound renewal seconds',
  'dhclient bound renewal seconds',
  'dhclient bound renewal seconds',
  'dhclient bound renewal seconds',
  'dhclient bound renewal seconds',
  'dhclient bound renewal seconds',
  'dhclient bound renewal seconds',
  'dhclient bound renewal seconds',
  'dhclient bound renewal seconds',
  'dhclient bound renewal seconds',
  'dhclient bound renewal seconds',
  'dhclient bound renewal seconds',
  'dhclient bound renewal seconds',
  'dhclient bound renewal seconds',
  'dhclient bound renewal seconds',
  'dhclient bound renewal seconds',
  'dhclient bound renewal seconds',
  'dhclient bound renewal seconds',
  'dhclient bound renewal seconds',
  'dhclient bound renewal seconds',
  'dhclient bound renewal seconds',
  'dhclient bound re

In [ ]:
trivial_logs = []
for key in list(my_dict.keys()):
    if len(my_dict[key])<1000:
        trivial_logs +=my_dict[key]

sentences = trivial_logs #tokenize sentences
t_nouns = [] #empty to array to hold all nouns

for sentence in sentences:
    tnoun = []
    for word in nltk.pos_tag(nltk.word_tokenize(str(sentence))):
        if (pos == 'VB' or pos == 'VBD' or pos == 'VBG' or pos == 'VBN' or pos == 'VBP' or pos == 'VBZ' or pos == 'JJ'):
             noun.append(word)
    nouns.append(noun)

## log_corpus for lda/nmf

In [161]:
import lda

no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features)
tfidf = tfidf_vectorizer.fit_transform(log_key_list)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features)
tf = tf_vectorizer.fit_transform(log_key_list)
tf_feature_names = tf_vectorizer.get_feature_names()

##########################

no_topics = 20

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda_model = lda.LDA(n_topics=no_topics, n_iter=15)
x_topics = lda_model.fit_transform(tf)

INFO:lda:n_documents: 1864
INFO:lda:vocab_size: 268
INFO:lda:n_words: 15106
INFO:lda:n_topics: 20
INFO:lda:n_iter: 15
/home/ubuntu/.local/lib/python3.5/site-packages/lda/utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):
INFO:lda:<0> log likelihood: -118273
INFO:lda:<10> log likelihood: -64567
INFO:lda:<14> log likelihood: -61923


In [162]:
from sklearn.manifold import TSNE

tsne_model = TSNE (n_components = 2, verbose = 1, random_state = 0, angle = 0.99, init='pca')
tsne_lda = tsne_model.fit_transform(x_topics)

# Clustering Visualization

import numpy as np
import bokeh.plotting as bp
from bokeh.plotting import save
from bokeh.models import HoverTool

n_top_words = 5 # the number of representing words for each cluster of documents

colormap = np.array([
  "#1f77b4", "#aec7e8", "#ff7f0e", "#ffbb78", "#2ca02c",
  "#98df8a", "#d62728", "#ff9896", "#9467bd", "#c5b0d5",
  "#8c564b", "#c49c94", "#e377c2", "#f7b6d2", "#7f7f7f",
  "#c7c7c7", "#bcbd22", "#dbdb8d", "#17becf", "#9edae5"])

_lda_keys = []

for i in range(x_topics.shape[0]):
    _lda_keys.append(x_topics[i].argmax())

    
num_example = len(x_topics)


topic_summaries = []
topic_word = lda_model.topic_word_  # all topic words
vocab = tf_vectorizer.get_feature_names()
for i, topic_dist in enumerate(topic_word):
   topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words + 1):-1] # get!
   topic_summaries.append(' '.join(topic_words)) # append!

color=colormap[_lda_keys][:num_example]
content = log_key_list[:num_example]
topic_key = _lda_keys[:num_example]


title = 'log files classification 100,000'


plot_lda = bp.figure(plot_width=1400, plot_height=1100,
                   title=title,
                   tools='pan,wheel_zoom,box_zoom,reset,hover,previewsave',
                   x_axis_type=None, y_axis_type=None, min_border=1)


ys=tsne_lda[:, 1]
xs=tsne_lda[:, 0]
source = bp.ColumnDataSource(data = dict(
               x = xs,
               y = ys,
               color=colormap[_lda_keys][:num_example],
               content = log_key_list[:num_example],
               topic_key = _lda_keys[:num_example]
                 ))
plot_lda.scatter(x ='x',
                y ='y',
               source=source,
               color ='color')

# randomly choose a conversation (within a topic) coordinate as the crucial words coordinate
topic_coord = np.empty((x_topics.shape[1], 2)) * np.nan
for topic_num in _lda_keys:
   if not np.isnan(topic_coord).any():
       break
   topic_coord[topic_num] = tsne_lda[_lda_keys.index(topic_num)]

# plot crucial words
for i in range(x_topics.shape[1]):
   plot_lda.text(topic_coord[i, 0], topic_coord[i, 1], [topic_summaries[i]])

# hover tools
hover = plot_lda.select(dict(type=HoverTool))
hover.tooltips = {"content": "@content - topic: @topic_key"}

# save the plot
save(plot_lda, title + '.html')

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1864 samples in 0.003s...
[t-SNE] Computed neighbors for 1864 samples in 0.174s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1864
[t-SNE] Computed conditional probabilities for sample 1864 / 1864
[t-SNE] Mean sigma: 0.000000
[t-SNE] KL divergence after 250 iterations with early exaggeration: 57.720024
[t-SNE] KL divergence after 1000 iterations: 0.367652


/home/ubuntu/.local/lib/python3.5/site-packages/bokeh/io/saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/home/ubuntu/.local/lib/python3.5/site-packages/bokeh/io/saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'/mnt/sdb/logs/nima/log files classification 100,000.html'